<a href="https://colab.research.google.com/github/Hurri301/Advanced-Computational-Techniques-for-Big-Imaging-and-Signal-Data/blob/main/Forest_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

In [3]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
import torchmetrics

In [18]:
# Load the dataset
df = pd.read_csv('/content/train.csv')

In [19]:
# Split the dataset into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['Cover_Type'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['Cover_Type'], random_state=42)

In [20]:
class ForestDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.features = dataframe.drop(columns=['Cover_Type']).values
        self.labels = dataframe['Cover_Type'].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx], dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.long) - 1  # Assuming classes are 1-indexed
        return x, y


In [21]:
# Create datasets and dataloaders
batch_size = 64

train_ds = ForestDataset(train_df)
val_ds = ForestDataset(val_df)
test_ds = ForestDataset(test_df)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [22]:
# Define the neural network
class Net(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [23]:
# Initialize the model, loss function, and optimizer
input_dim = train_ds.features.shape[1]
num_classes = 7
learning_rate = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = Net(input_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [24]:
# Training and validation loop
num_epochs = 20

for epoch in range(num_epochs):
    net.train()
    for inputs, labels in train_dl:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    net.eval()
    val_loss = 0
    val_accuracy = Accuracy(task="multiclass", num_classes=num_classes).to(device)
    for inputs, labels in val_dl:
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_accuracy.update(outputs, labels)

    val_loss /= len(val_dl)
    val_acc = val_accuracy.compute()
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/20, Validation Loss: 0.8611, Validation Accuracy: 0.6213
Epoch 2/20, Validation Loss: 0.7282, Validation Accuracy: 0.6858
Epoch 3/20, Validation Loss: 0.6939, Validation Accuracy: 0.7062
Epoch 4/20, Validation Loss: 0.6557, Validation Accuracy: 0.7211
Epoch 5/20, Validation Loss: 0.6434, Validation Accuracy: 0.7216
Epoch 6/20, Validation Loss: 0.6519, Validation Accuracy: 0.7067
Epoch 7/20, Validation Loss: 0.6133, Validation Accuracy: 0.7370
Epoch 8/20, Validation Loss: 0.6234, Validation Accuracy: 0.7337
Epoch 9/20, Validation Loss: 0.5923, Validation Accuracy: 0.7293
Epoch 10/20, Validation Loss: 0.5868, Validation Accuracy: 0.7387
Epoch 11/20, Validation Loss: 0.5896, Validation Accuracy: 0.7409
Epoch 12/20, Validation Loss: 0.5672, Validation Accuracy: 0.7657
Epoch 13/20, Validation Loss: 0.5594, Validation Accuracy: 0.7602
Epoch 14/20, Validation Loss: 0.5680, Validation Accuracy: 0.7470
Epoch 15/20, Validation Loss: 0.5576, Validation Accuracy: 0.7668
Epoch 16/20, Valida

In [25]:
# Test the model
net.eval()
test_accuracy = Accuracy(task="multiclass", num_classes=num_classes).to(device)
confusion_matrix = ConfusionMatrix(task="multiclass", num_classes=num_classes).to(device)
for inputs, labels in test_dl:
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs = net(inputs)
        test_accuracy.update(outputs, labels)
        confusion_matrix.update(outputs, labels)

test_acc = test_accuracy.compute()
conf_matrix = confusion_matrix.compute()

print(f'Test Accuracy: {test_acc:.4f}')
print(f'Confusion Matrix:\n{conf_matrix.cpu().numpy()}')


Test Accuracy: 0.7609
Confusion Matrix:
[[154  68   0   0   4   0  36]
 [ 52 176   3   0  16  10   3]
 [  0   6 138  18  12  78   0]
 [  0   0  12 236   0  17   0]
 [  4  30   5   0 208   4   0]
 [  0   4  26   7   5 220   0]
 [ 12   2   0   0   0   0 249]]
